In [ ]:
import pandas as pd
import numpy as np
import rtsvg
rt = rtsvg.RACETrack()
import time
import csv
import json
import os
from ollama import chat
from ollama import ChatResponse
from pydantic import BaseModel
# model = 'deepseek-r1:32b'
model = 'qwq' # 'llama3.2'
def promptModel(prompt, context_window_size=8192):
    response: ChatResponse = chat(model=model, messages=[{ 'role': 'user', 'content': prompt,},], options={'num_ctx': context_window_size})
    return response['message']['content']
_results_ = promptModel('What is 55*3?  Return a single number.') # force the model load
import random
import copy
def randomizeOrder(_lines_):
    _lines_ = copy.deepcopy(_lines_)
    new_lines = []
    while len(_lines_) > 0:
        _index_ = random.randint(0, len(_lines_) - 1)
        new_lines.append(_lines_.pop(_index_))
    return new_lines
social_media_lines = []
_mc3_dir_ = '../../../data/2014_vast/MC3'
_files_   = ['csv-1700-1830.csv','csv-1831-2000.csv','csv-2001-2131.csv']
_dfs_     = []
for _file_ in _files_:
    _file_ = _mc3_dir_ + '/' + _file_
    with open(_file_, encoding='iso-8859-1') as f:
        _lines_ = f.readlines()
        social_media_lines.extend(_lines_)
    _header_, _for_df_ = None, {}
    with open(_file_, 'r', encoding='iso-8859-1') as file:
        csv_reader = csv.reader(file)
        for row in csv_reader:
            if _header_ is None:
                _header_ = row
                for _col_ in _header_: _for_df_[_col_] = []
            else:
                for _index_ in range(len(row)): 
                    _for_df_[_header_[_index_]].append(row[_index_])
    _df_ = pd.DataFrame(_for_df_)
    _df_['original_line'] = _lines_[1:]
    _dfs_.append(_df_)

df_social_media                 = pd.concat(_dfs_)
df_social_media['newman_rally'] = df_social_media['message'].str.lower().str.contains('newman') & df_social_media['message'].str.lower().str.contains('rally')

newman_lines, non_newman_lines = [], []
for i in range(len(social_media_lines)):
    if   'newman' in social_media_lines[i].lower() and 'rally' in social_media_lines[i].lower():  newman_lines    .append(social_media_lines[i])
    elif 'newman' in social_media_lines[i].lower():                                               pass # just make sure that newman isn't in the dataset at all...
    else:                                                                                         non_newman_lines.append(social_media_lines[i])
print(f'{len(social_media_lines)=} {len(newman_lines)=} {len(non_newman_lines)=}')

In [ ]:
def createNeedlePrompt(wo_newman, w_newman):
    _lines_  = []
    _prompt_ = f'Which doctor spoke at the POK Rally?  Use the following social media JSON posts to answer the question.  Respond with their name in JSON format.  If there is no answer, respond with "None".\n\n'
    for i in range(w_newman):
        _df_   = None
        while _df_ is None or len(_df_) == 0:
            _line_ = random.choice(newman_lines)
            _df_   = df_social_media[df_social_media['original_line'] == _line_].reset_index()
        _lines_.append(str(json.dumps({'post':json.dumps(_df_.iloc[0]['message'])}))+'\n')
    for i in range(wo_newman):
        _df_ = None
        while _df_ is None or len(_df_) == 0:
            _line_ = random.choice(non_newman_lines)
            _df_   = df_social_media[df_social_media['original_line'] == _line_].reset_index()
        _lines_.append(str(json.dumps({'post':json.dumps(_df_.iloc[0]['message'])}))+'\n')
    _lines_ = randomizeOrder(_lines_)
    return _prompt_ + ''.join(_lines_)

class RallySpeaker(BaseModel):
    name: str

_tuples_ = []
for i in range(10, 1000, 100): # len(newman_lines)+len(non_newman_lines), 50):
    for j in range(0, 2): # Inclusions of a Newman Line
        for k in range(1): # Samples
            _tuples_.append((i, j, k))
len(_tuples_)

In [ ]:
_lu_ = {'time_taken':[], 'name':[], 'valid':[], 'wo_newman':[], 'w_newman':[], 'prompt':[], 'prompt_eval_count':[], 'eval_count':[], 'message_content':[], 'prompt_length':[]}
for _index_ in range(len(_tuples_)):
    i, j, k = _tuples_[_index_]
    _prompt_ = createNeedlePrompt(i, j)
    t0 = time.time()
    response: ChatResponse = chat(model=model, messages=[{ 'role': 'user', 'content':  _prompt_,},], format=RallySpeaker.model_json_schema(),options={'num_ctx': 32768})
    t1 = time.time()
    rally_speaker = RallySpeaker.model_validate_json(response['message']['content'])
    _lu_['time_taken'].append(t1 - t0)
    _lu_['name'].append(rally_speaker.name)
    _lu_['wo_newman'].append(i)
    _lu_['w_newman'].append(j)
    _lu_['prompt'].append(_prompt_)
    _lu_['prompt_eval_count'].append(response.prompt_eval_count)
    _lu_['eval_count'].append(response.eval_count)
    _lu_['message_content'].append(response.message.content)
    _lu_['prompt_length'].append(len(_prompt_))
    if    j == 0: _lu_['valid'].append(rally_speaker.name.lower() == 'none')
    else:         _lu_['valid'].append('newman' in rally_speaker.name.lower())
    if _index_%50 == 0: print(f'{_index_}/{len(_tuples_)}',end='')
    else:               print('.', end='')

df_niah = pd.DataFrame(_lu_)
params = {'df':df_niah, 'dot_size':'vary', 'max_dot_size':3.0, 'color_by':'valid'}
rt.tile([rt.xy(x_field='prompt_eval_count', y_field='time_taken',        **params),
         rt.xy(x_field='eval_count',        y_field='time_taken',        **params),
         rt.xy(x_field='prompt_length',     y_field='time_taken',        **params),
         rt.xy(x_field='prompt_length',     y_field='prompt_eval_count', **params)],spacer=10)

In [ ]:
# df_niah.to_parquet(_mc3_dir_ + '/20250305_1940_niah_deepseek_32b_context_window_issues.parquet')
rt.smallMultiples(df=df_niah, sm_type='histogram', category_by='w_newman', sm_params={'bin_by':'name'}, color_by='valid', w=1536, h=512)